<a href="https://colab.research.google.com/github/Rtniewi/kcwiertniewicz-IDS/blob/main/GenAI/HW5/LSTM_2_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

//***Katrina Cwiertniewicz
//*** CSC 330
//11/22/2024
//Assignment 5: Text Generation Using LSTM on Project Gutenberg Training Data
####The purpose of this assignment is to develop an LSTM model that generates text. The goal is to produce coherent and stylistically relevant text based on prompts.

In [ ]:
import numpy as np
import json
import re
import string

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses

## 0. Parameters <a name="parameters"></a>

In [ ]:
VOCAB_SIZE = 20049
MAX_LEN = 500
EMBEDDING_DIM = 100
N_UNITS_LAYER_1 = 64
N_UNITS_LAYER_2 = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25

## 1. Load the data <a name="load"></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pwd

'/content'

In [ ]:
import requests
import os


# List of files for additional texts (e.g., different Edgar Allan Poe Works)
file_paths = [
  os.path.join('/content/drive/MyDrive/text/The_Tell_Tale_Heart.txt'),                        # The Tell Tale Heart
  os.path.join('/content/drive/MyDrive/text/The_Cask_of_Amontillado.txt'),                    # The Cask of Amontillado
  os.path.join('/content/drive/MyDrive/text/The_Raven.txt'),                                  # The Raven
  os.path.join('/content/drive/MyDrive/text/The_Masque.txt'),                                 # The Masque of the Red Death
  os.path.join('/content/drive/MyDrive/text/Annabel_Lee.txt'),                                # Annabel Lee
  os.path.join('/content/drive/MyDrive/text/Lenore.txt'),                                     # Lenore
  os.path.join('/content/drive/MyDrive/text/The_Bells.txt'),                                  # The Bells
  os.path.join('/content/drive/MyDrive/text/The_Black_Cat.txt'),                              # The Black Cat
  os.path.join('/content/drive/MyDrive/text/The_Fall_of_the_House_of_Usher.txt'),             # The Fall of the House of Usher
  os.path.join('/content/drive/MyDrive/text/The_Oval_Portrait.txt'),                          # The Oval Portrait
  os.path.join('/content/drive/MyDrive/text/The_Pit_and_the_Pendulum.txt'),                   # The Pit and the Pendulum
  os.path.join('/content/drive/MyDrive/text/The_Premature_Burial.txt'),                       # The Premature Burial
  os.path.join('/content/drive/MyDrive/text/The_Narrative_of_Arthur_Gordon.txt'),             # The Narrative of Arthur Gordon Pym of Nantucket
  os.path.join('/content/drive/MyDrive/text/Al_Aaraaf.txt'),                                  # Al Aaraaf
  os.path.join('/content/drive/MyDrive/text/Alone.txt'),                                      # Alone
  os.path.join('/content/drive/MyDrive/text/Eureka.txt'),                                     # Eureka
  os.path.join('/content/drive/MyDrive/text/The_Facts_in_the_Case_of_M._Valdemar.txt'),       # The Facts in the Case of M.Valdemar
  os.path.join('/content/drive/MyDrive/text/A_Descent_into_the_Maelstrom.txt'),               # A Descent into the Maelstrom
  os.path.join('/content/drive/MyDrive/text/William_Wilson.txt'),                             # William Wilson
  os.path.join('/content/drive/MyDrive/text/Berenice.txt'),                                   # Berenice
  os.path.join('/content/drive/MyDrive/text/The_Gold_Bug.txt'),                               # The Gold Bug
  os.path.join('/content/drive/MyDrive/text/The_Murders_of_Rue_Morgue.txt'),                  # The Murders in the Rue Morgue
  os.path.join('/content/drive/MyDrive/text/The_Mystery_of_Marie_Roget.txt'),                 # The Mystery of Marie Roget
  os.path.join('/content/drive/MyDrive/text/The_Purloined_Letter.txt'),                       # The Purloined Letter
  os.path.join('/content/drive/MyDrive/text/Von_Kempelen_and _his_Discovery.txt'),            # Von Kempelen and His Discovery
  os.path.join('/content/drive/MyDrive/text/Island_of_the_Fay.txt'),                          # Island of the Fay
  os.path.join('/content/drive/MyDrive/text/Mesmeric_Revelation.txt'),                        # Mesemeric Revelation
  os.path.join('/content/drive/MyDrive/text/Silence_A_Fable.txt'),                            # Silence a Fable
  os.path.join('/content/drive/MyDrive/text/MS._Found_in_a_Bottle.txt'),                      # MS. Found in a Bottle
  os.path.join('/content/drive/MyDrive/text/The_Thousand_and_Second_Tale_of_Scherezade.txt'), # The Thousand and Second Tale of Scherezade
  os.path.join('/content/drive/MyDrive/text/The_Unparalleled_Adventure.txt'),                 # The Unparalleled Adventure of One Hans Pfaall
  os.path.join('/content/drive/MyDrive/text/The_Assignation.txt'),                            # The Assignation
  os.path.join('/content/drive/MyDrive/text/The_Imp.txt'),                                    # The Imp of the Perverse
  os.path.join('/content/drive/MyDrive/text/The_Domain_of_Arnheim.txt'),                      # The Domain of Arnheim
  os.path.join('/content/drive/MyDrive/text/The_Assignation.txt'),                            # Landor's Cottage
  os.path.join('/content/drive/MyDrive/text/Morella.txt'),                                    # Morella
  os.path.join('/content/drive/MyDrive/text/Ligeia.txt'),                                     # Ligeia
  os.path.join('/content/drive/MyDrive/text/King_Pest.txt'),                                  # King Pest
  os.path.join('/content/drive/MyDrive/text/A_Tale_of_the_Ragged_Mountains.txt'),             # A Tale of the Ragged Mountains
  os.path.join('/content/drive/MyDrive/text/The_Spectacles.txt'),                             # The Spectacles
  os.path.join('/content/drive/MyDrive/text/Philosophy_of_Furniture.txt'),                    # The Philosophy of Furniture
  os.path.join('/content/drive/MyDrive/text/The_Devil_in_Belfry.txt'),                        # The Devil in the Belfry
  os.path.join('/content/drive/MyDrive/text/Bon_Bon.txt'),                                    # Bon-Bon
  os.path.join('/content/drive/MyDrive/text/Some_Words_with_a_Mummy.txt')                     # Some Words with a Mummy


]

# Initialize an empty string to hold all text
all_text = ""

# Download each text file and append to all_text
for file_path in file_paths:
  with open(file_path, 'r') as file:
    content = file.read()
    text = content
    all_text += text + "\n\n"  # Separate texts by newlines

# Save combined text to a single file
  with open('/content/combined_poe.txt', "w", encoding="utf-8") as file:
    file.write(all_text)


In [ ]:
# Count the words of text
with open('/content/combined_poe.txt', "r", encoding="utf-8") as file:
  file_content = file.read()
  words = file_content.split()
  n_words = len(words)
print(f"{n_words} words loaded")

326500 words loaded


In [ ]:
# Example Sentence of First Ten Words
example_sentence = words[:10]
print(f"Example Sentence: {example_sentence}")

Example Sentence: ['True!—nervous—very,', 'very', 'dreadfully', 'nervous', 'I', 'had', 'been', 'and', 'am;', 'but']


## 2. Tokenise the data

In [ ]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


with open("combined_poe.txt", "r", encoding="utf-8") as file:
    text_data = [pad_punctuation(line) for line in file]

In [ ]:
example_date = text_data[30]
print(example_date)

Ha ! —would a madman have been so wise as this ? And then , when my



In [ ]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [ ]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [ ]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

# Length of Vocabulary
print(f"Length of Vocabulary: {len(vocab)}")

Length of Vocabulary: 20049


In [ ]:
# Display some token:word mappings
for i, word in enumerate(vocab[10:30]):
    print(f"{i}: {word}")

0: -
1: i
2: that
3: _
4: it
5: was
6: as
7: with
8: which
9: at
10: my
11: is
12: had
13: ;
14: we
15: this
16: for
17: by
18: not
19: be


In [ ]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(text_data)
print(example_tokenised.numpy())

[[  326    47 11929 ...     0     0     0]
 [   33   329    74 ...     0     0     0]
 [13630 18134 10712 ...     0     0     0]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]]


## 3. Create the Training Set

In [ ]:
# Create the training set of text and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

## 4. Build the LSTM <a name="build"></a>

In [ ]:
inputs = layers.Input(shape=(None,), dtype="int32")
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS_LAYER_1, return_sequences=True)(x)
x = layers.LSTM(N_UNITS_LAYER_2, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, None, 100)           │       2,004,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, None, 64)            │          42,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, None, 128)           │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, None, 20049)         │       2,586,321 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,732,277 (18.05 MB)

 Trainable params: 4,732,277 (18.05 MB)

 Non-trainable params: 0 (0.00 B)

## 5. Train the LSTM <a name="train"></a>

In [ ]:
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)

In [ ]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }  # <1>

    def sample_from(self, probs, temperature):  # <2>
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]  # <3>
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0: # <4>
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)  # <5>
            sample_token, probs = self.sample_from(y[0][-1], temperature)  # <6>
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # <7>
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("Text:", max_tokens=1000, temperature=1.0)

In [ ]:
# Tokenize starting prompt

text_generator = TextGenerator(vocab)

In [ ]:
lstm.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[text_generator],
)

Epoch 1/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - loss: 1.1551
generated text:
Text: corn crest to from the the an entirely 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 211s 199ms/step - loss: 1.1543
Epoch 2/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - loss: 0.1442
generated text:
Text: suddenly the herself hatred pig—which , over this w 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 209s 202ms/step - loss: 0.1442
Epoch 3/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - loss: 0.1364
generated text:
Text: blasted male over we three as , among found in then , here quivering to 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 210s 203ms/step - loss: 0.1364
Epoch 4/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - loss: 0.1296
generated text:
Text: opinion the deck and visions , humming . it was certain 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 210s 203ms/step - loss: 0.1296
Epoch 5/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - loss: 0.1266
generated text:
Text: window in them , i heavy you may earnest inconceivable up _ 

## 6. Generate text using the LSTM

In [ ]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [ ]:
# Prompt 1: From "The Raven"    Temperature=0.1
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=500, temperature=0.1
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. in the armory . i had no doubt of the nature of the most singular and shrunken frame . 



In [ ]:
print_probs(info, vocab)

Streaming output truncated to the last 5000 lines.
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. to all now , — the material lived — not full , and gently magnetism to seek a queen , and overflowing in hand , along the bosom of the car . after my years , with
the:   	33.16%
a:   	23.67%
my:   	4.94%
many:   	4.48%
some:   	3.2%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber 

In [ ]:
# Prompt 1: From "The Raven"    Temperature=0.5
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=1000, temperature=0.5
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. , and , in the typography of meditation — and the mists of the world in the nature of the room , the speculative objects of the finer , a very singular and gloomy and disease , and the grave of the mental waters in the fire . 



In [ ]:
print_probs(info, vocab)

Streaming output truncated to the last 5000 lines.
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. to all now , — the material lived — not full , and gently magnetism to seek a queen , and overflowing in hand , along the bosom of the car . after my years , with
the:   	33.16%
a:   	23.67%
my:   	4.94%
many:   	4.48%
some:   	3.2%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber 

In [ ]:
# Prompt 1: From "The Raven"    Temperature=1.0
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=1000, temperature=1.0
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. — and thereunto , ’ which , with curtains , which was in vain and troubled every jane syllable in the stowage — but demanded it in suspense upon some anxiety of trance ? whose tous disease had her case , even burned from red fountains , they were who saw myself for a very fresh nature and seventh pages ; the glassy character of a wisdom , when abrupt and cards like a spade ascendancy . her time to seek in a thin and tumultuous coffin brought into the armory observations in his father as i might have been i seen , black and metaphysical , forming for her single grey spirit of ambiguity , and addicted before few saloo

In [ ]:
print_probs(info, vocab)


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.
in:   	18.53%
,:   	13.39%
to:   	7.05%
with:   	6.63%
by:   	6.28%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. —
and:   	29.16%
i:   	15.23%
not:   	6.21%
the:   	6.05%
with:   	3.16%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and 

In [ ]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=0.1
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=0.1
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. . i had no doubt , in the manner of the library , and , in the first instance , i had no more trouble to the mind of the library , and , in the most remote of the most common and pernicious vagaries of the library , and , in the first instance , i had no more trouble to the grave of the library . i had no more of the most singular and exciting manner . 



In [ ]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
.:   	99.91%
,:   	0.09%
;:   	0.0%
—:   	0.0%
-:   	0.0%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. .
i:   	99.49%
in:   	0.39%
the:   	0.09%
:   	0.02%
there:   	0.0%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acute

In [ ]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=0.5
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=0.5
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. . i strived to decypher them , and with the grey of the library , i saw no attention of the library , and , in the most echo of my own , and in the pale , and the unintelligible and of the library , and in the antechamber a servant and lingered , and the grey of the winds , and in the sense of the most appalling and terror - - the speculative eye . the waters of the day — and the heart of the sound , and i struggled in vain , and many words were of the characteristic saloon of the land of the library , and , with a wild and desolate objects of the most shrubberies , and directed the victim of the lib

In [ ]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
.:   	76.65%
,:   	18.93%
;:   	4.19%
—:   	0.08%
-:   	0.05%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. .
i:   	47.14%
in:   	15.59%
the:   	11.63%
:   	8.77%
there:   	2.99%
--------


PROMPT: And have I not told you that what you mistake for madness is but ove

In [ ]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=1.0
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=1.0
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. . some varieties was grown against the day a soft days , just about her chin , and was off , until i wonder how seldom very deed — i am saying in the monomania , but of narrow dreams buried from at silence like an unhappy — of hues could not more dawned in our note and smiled . 



In [ ]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
.:   	51.29%
,:   	25.49%
;:   	11.99%
—:   	1.61%
-:   	1.28%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. .
i:   	15.39%
in:   	8.85%
the:   	7.64%
:   	6.64%
there:   	3.88%
--------


PROMPT: And have I not told you that what you mistake for madness is but over

In [ ]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=0.1
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=0.1
)


generated text:
There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  and frivolous . i had not been told to the grave of the library . i had no more unequivocally and in the library . i had no doubt of the nature of the singular character of the library - heartedness and piercing condition , and the embers of the cloister . i had no doubt of the nature of the most singular and obstinate and meditation — the most distressing and shrunken frame . 



In [ ]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. 
.:   	93.86%
and:   	6.13%
,:   	0.0%
as:   	0.0%
;:   	0.0%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  and
dismay:   	78.06%
frivolous:   	21.94%
physical:   	0.0%
palpably:   	0.0%
obstinately:   	0.0%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so se

In [ ]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=0.5
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=0.5
)


generated text:
There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  . i could not fail to the fact , of the nature of the immoderate and shrunken , — the day who has no design of no such existence . 



In [ ]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. 
.:   	53.63%
and:   	31.08%
,:   	6.55%
as:   	6.25%
;:   	1.01%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  .
:   	56.51%
i:   	26.51%
the:   	6.75%
and:   	3.56%
in:   	2.29%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a charact

In [ ]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=1.0
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=1.0
)


generated text:
There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  at among the most abstruse after my recollections . they could you will attempt to me i humble , and the unintelligible word of my musings pocket . had no such traces of a newly husky was distinct on antique terror - honored and in the gallery that bursting died to springs it frivolous . — i good echoed all that words , eight and anon , but such follies . probing 



In [ ]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. 
.:   	30.5%
and:   	23.22%
,:   	10.66%
as:   	10.42%
;:   	4.2%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  at
the:   	32.52%
all:   	5.86%
a:   	4.94%
:   	4.47%
least:   	3.3%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a chara